# Preliminaries

R. N. Modification of the original code from Prof Avi Rosenfeld.

Note: 2 queens and 3 queens do not have a solution.

This is the notebook version of the code. I will use this to explain the homework.  I used parts of the code from: https://www.sanfoundry.com/python-program-solve-n-queen-problem-without-recursion/

As we did in class, we will represent the board as a one-dimensional array "columns" where each position  in the array is the row value.

So if the array is: [1, 3, 0, 2], then the  queen in the first row is in column 1 (columns are labeled from 0--3), the queen in the second row is in column 3 (the last column), the queen in the third row is in column 0 and the  queen in the last row is in column 2.

Note that sometimes we will have a partial board like [1,3] which means that only the first 2 rows have queens placed, and the remaining row placements are still to be ascertained.



## Random board placement
Let's setup one iteration of the British Museum algorithm-- we'll put down n=8 queens randomly.

In [4]:

import random
def place_n_queens(n):
    assert n >= 0
    columns = [random.randrange(0,n) for x in range(n)]
    return columns


def displayBoard(columns):
    if not columns:
        return
    n = len(columns)
    print(columns)
    for rowVal in range(n):
        rowStr = ["." for x in range(n)]
        rowStr[columns[rowVal]] = '♛'
        print("  ".join(rowStr))
    print()




## Solution validity check
We also need some way to determine if such a board position is valid, which we can check row by row as we traverse the columns array to check for conflicts with the queen placements in the previous rows.

In [6]:
# check if the next row queen can be placed in column col
def is_col_place_in_next_row_valid(col, columns):

    row = len(columns) # next row

    def check_no_conflicts(col, row, queen_col, queen_row):
      return (col != queen_col) and (queen_col - queen_row != col - row) and (queen_col + queen_row != col + row)

    # return True if all previous queen placements do not conflict with the proposed (row, col) position
    return all([check_no_conflicts(col, row, queen_col, queen_row) for queen_row, queen_col in enumerate(columns)])

# use the helper function to check for the validity of the full board
def is_valid_solution(size, columns):
    return len(columns) == size and all([is_col_place_in_next_row_valid(col, columns[:row]) for row, col in enumerate(columns)])

# is some random placement  a valid solution
columns = place_n_queens(8)
print("proposed solution:")
displayBoard(columns)
print(f"is this a valid solution?: {is_valid_solution(8, columns)}")

proposed solution:
[2, 1, 0, 3, 2, 0, 1, 3]
.  .  ♛  .  .  .  .  .
.  ♛  .  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .
.  .  ♛  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .

is this a valid solution?: False


Since this random arrangement of queens is of course unlikely to be a solution, we can iterate this till we get a solution.  This is the British Museum algorithm, which is obviously pretty naive approach, but easy to implement.

# Algorithm testing and measurement harness
First though, we introduce a general-purpose harness for running any n-queens solution algorithm.

We assume that the algorithm is  implemented with the function signature:

```
columns, num_iterations, num_moves, converged=solve_algorithm(size)```

Here, size is the n in n-queens, columns is the final solution which is a converged solution if converged = True, and num_iterations and num_moves are measures of the efficiency of the algorithm (One can think of num_iterations as the number of state transitions, and num_moves as the number of queens moved cumulatively over these state transition).   

For example if every state transition only involved 1 queen move, then num_iterations = num_moves, but as in the British museum algorithm if each state transition involves 8 queen placements then num_moves = 8 * num_iterations.

The solve_nqueens harness method takes as arguments the function reference to the n-queens solution algorithm above, size (which is the n in n-queens) and ntrials which can be greater than 1 for  algorithms that have a random component, e.g. random initialization or random search (if ntrials > 1, then the averaged set of results over multiple trials is returned).

In [7]:
def solve_nqueens(solve_algorithm, size = 8,  ntrials = 1):

  num_iterations_list, num_moves_list, convergence_list = [], [], []
  for i in range(0, ntrials):
    columns, num_iterations, num_moves, converged=solve_algorithm(size)
    num_iterations_list.append(num_iterations)
    num_moves_list.append(num_moves)
    convergence_list.append(converged)
    print(f"is converged: {converged}")
    print(f"is valid solution obtained (check): {is_valid_solution(size, columns)}")
    print(f"number of iterations: {num_iterations}")
    print(f"number of moves: {num_moves}")
    if converged:
        print("display the results")
        displayBoard(columns)

  if ntrials > 1:
    print(f"average number of iterations: {sum(num_iterations_list)/len(num_iterations_list)}")
    print(f"average number of moves: {sum(num_moves_list)/len(num_moves_list)}")
    print(f"average convergence: {sum(convergence_list)/len(convergence_list)}")

# British Museum algorithm

Now what?  Can you implement the British Museum Algorithm?  How many moves and iterations did it take to solve the 4 queens problem?  

How many moves/iterations did it take to solve the 8 queens (if at all)?

In [8]:

def solve_britishmuseum(size):

  MAX_NUMBER_OF_ITERATIONS = 1000000
  number_of_iterations, number_of_moves, converged = 0, 0, False

  while not converged and number_of_iterations < MAX_NUMBER_OF_ITERATIONS:
    number_of_iterations += 1
    columns = place_n_queens(size)
    number_of_moves += 8
    if is_valid_solution(size,columns):
      converged = True

  return columns, number_of_iterations, number_of_moves, converged

In [9]:

solve_nqueens(solve_britishmuseum, size = 8,  ntrials = 1)


is converged: True
is valid solution obtained (check): True
number of iterations: 258768
number of moves: 2070144
display the results
[4, 1, 3, 5, 7, 2, 0, 6]
.  .  .  .  ♛  .  .  .
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .
.  .  .  .  .  ♛  .  .
.  .  .  .  .  .  .  ♛
.  .  ♛  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .



# DFS algorithm with backtracking

The BM approach is not very efficient, so we'll write a simple DFS search that places the n queens row by row with backtracking:

First some helper functions to tentatively place and remove a queen from the next row.

1. place_next_row

2. remove_current_row

These functions will be called after using the is_col_place_in_next_row_valid function given earlier to ensure that a new row placement is valid

In [12]:
def place_next_row(col, columns):
    columns.append(col)

def remove_current_row(columns):
    if len(columns) > 0:
        return columns.pop()
    return -1

We can now write the desired dfs code by starting with a blank board and placing the queens in one row at a time, starting with the first col in the first row.



In [10]:
def solve_dfs(size):
    columns = []
    MAX_NUMBER_OF_ITERATIONS = 1000
    number_of_iterations, number_of_moves, converged = 0, 0, False

    row = 0 # current row
    col = 0 # always start each row at leftmost col
    # iterate over rows of board
    while True and number_of_iterations < MAX_NUMBER_OF_ITERATIONS :
        #this loop places a queen in next row
        number_of_iterations += 1
        while col < size:
            if is_col_place_in_next_row_valid(col, columns):
                place_next_row(col, columns)
                number_of_moves += 1
                # print(f"placed: row: {row}, col: {col}")
                row += 1
                col = 0
                break
            else:
                # print(f"not placed: row: {row}, col: {col}")
                col += 1


        # could not find an open col in this row (backtrack),  or board is full (converged)
        if (col == size or row == size):
            number_of_iterations+=1
            # if board is full, we have a solution
            if row == size:
                print("I did it! Here is my solution")
                display(columns)
                converged = True
                return columns, number_of_iterations, number_of_moves, converged
            # else couldn't find a solution so need to backtrack
            # print("start to backtrack ... ")
            prev_col = remove_current_row(columns)
            if (prev_col == -1): # backtracked past column 1
                print("There are no solutions")
                converged = False
                return columns, number_of_iterations, number_of_moves, converged
            # retry previous row again
            row -= 1
            # start to now check at col = (1 + value of prev_column in the row)
            col = 1 + prev_col



Does the DFS algorithm get to 30 (not easily at all!).  To show this check the values for num_iterations and number_moves for values of 10, 20 and 30.  Feel free to stop the DFS algorithm after 10,000,000 iterations!


In [13]:
solve_nqueens(solve_dfs, size = 8,  ntrials = 1)

I did it! Here is my solution


[0, 4, 7, 5, 2, 6, 1, 3]

is converged: True
is valid solution obtained (check): True
number of iterations: 324
number of moves: 113
display the results
[0, 4, 7, 5, 2, 6, 1, 3]
♛  .  .  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  .  .  ♛  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .



# Hill climbing algorithm with heuristic repair
Now implement your Hill Climbing Heuristic Repair Algorithm and check the values for num_iterations and number_moves for values of 10, 20 and 30.

In this algorithm there are no partial boards, and all the board positions have all the n queens placed on the board, one in each row (e.g. by random placement).   The algorithm then attempts to improve the board quality by moving any one queen to another position in the same row which leads to a minimum number of conflicts. Therefore each board position evaluation requires evaluating potentially n^2 new states that can be transitioned to, and the final transition is to the best possible successor state.

First some useful helper functions to count the number of conflicts for the queen in each row, and total number of conflicts in the entire board (which is the sum of the individual row conflicts divided by 2).

Note that the computation of the conflicts is very inefficient and O(n^2) since each configuration is evaluated from scratch.  A more efficient incremental evaluation with O(1) complexity is possible, and will be implemented in future work.

Also note that the hill climbing algorithm can lead to a local mininum which is not a valid solution.   We use a heuristic where if there is not improvement after MAX_ITERATIONS_WITHOUT_IMPROVEMENT, then the algorithm restarts from a new random initial configuration.  

The implemented algorithm can be speeded up in a few ways: more efficient board evaluation, random moves that don't necessarily pick the best successor state (in order to avoid local minima) etc.

In [17]:
def count_row_conflicts(row, columns):
  assert row < len(columns)
  row_conflicts = 0
  col = columns[row]
  for queen_row, queen_col in enumerate(columns):
    if queen_row != row:
      row_conflicts +=  (col == queen_col) + (queen_col - queen_row == col - row) + (queen_col + queen_row == col + row)
  return row_conflicts

def count_conflicts(columns):
  return sum([count_row_conflicts(row, columns) for row in range(len(columns))])/2

# some testing code
def show_count_row_conflicts():
  print("display a board with no conflicts")
  columns = [5, 1, 6, 0, 2, 4, 7, 3] # solution
  displayBoard(columns)
  for row in range(len(columns)):
    print(f"row: {row}, row conflicts: {count_row_conflicts(row, columns)}")
  assert all([count_row_conflicts(row, columns) == 0 for row in range(len(columns))])

  print()
  print("display a board with conflicts")
  columns = [5, 1, 0, 6, 2, 4, 7, 3] # not a solution

  displayBoard(columns)
  for row in range(len(columns)):
    print(f"row: {row}, row conflicts: {count_row_conflicts(row, columns)}")
  assert not all([count_row_conflicts(row, columns) == 0 for row in range(len(columns))])


def test_count_conflicts():
  columns = [5, 1, 6, 0, 2, 4, 7, 3] # solution
  assert count_conflicts(columns) == 0

  print()
  columns = [5, 1, 0, 6, 2, 4, 7, 3] # not a solution
  assert count_conflicts(columns) == 3


show_count_row_conflicts()
test_count_conflicts()


display a board with no conflicts
[5, 1, 6, 0, 2, 4, 7, 3]
.  .  .  .  .  ♛  .  .
.  ♛  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
♛  .  .  .  .  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  ♛  .  .  .  .

row: 0, row conflicts: 0
row: 1, row conflicts: 0
row: 2, row conflicts: 0
row: 3, row conflicts: 0
row: 4, row conflicts: 0
row: 5, row conflicts: 0
row: 6, row conflicts: 0
row: 7, row conflicts: 0

display a board with conflicts
[5, 1, 0, 6, 2, 4, 7, 3]
.  .  .  .  .  ♛  .  .
.  ♛  .  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  .  ♛  .  .  .  .  .
.  .  .  .  ♛  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  ♛  .  .  .  .

row: 0, row conflicts: 0
row: 1, row conflicts: 1
row: 2, row conflicts: 2
row: 3, row conflicts: 1
row: 4, row conflicts: 1
row: 5, row conflicts: 1
row: 6, row conflicts: 0
row: 7, row conflicts: 0



In [18]:
def solve_hillclimbing(size):

  MAX_NUMBER_OF_ITERATIONS = 1000
  MAX_ITERATIONS_WITHOUT_IMPROVEMENT = 10
  number_of_iterations, number_of_moves, converged = 0, 0, False

  columns = place_n_queens(size)
  min_conflict_count = count_conflicts(columns)
  print(f"initial conflict count: {min_conflict_count}")

  while not converged and number_of_iterations < MAX_NUMBER_OF_ITERATIONS:
    number_of_iterations += 1

    #check all possible moves and pick the best
    best_row, best_col = -1, -1
    for row in range(size):
      col = columns[row]
      for new_col in range(size):
        columns[row] = new_col # swap to new state
        conflict_count = count_conflicts(columns)
        if conflict_count < min_conflict_count:
          min_conflict_count = conflict_count
          best_row, best_col = row, new_col
          improvement_count = 0  # reset counter for tracking no improvement per iteration

        columns[row] = col  # swap back to old state and continue

    # replace with best solution at the end of the scan
    # print(f"min_conflict_count: {min_conflict_count}")
    if best_row != -1: # better solution found
      columns[best_row] = best_col
      number_of_moves += size*(size-1) # each open col position is evaluated for each row
    else:  # no better solution found
      improvement_count += 1
      if improvement_count > MAX_ITERATIONS_WITHOUT_IMPROVEMENT: # restart with another random configuration
          columns = place_n_queens(size)
          min_conflict_count = count_conflicts(columns)

    converged = min_conflict_count == 0

  return columns, number_of_iterations, number_of_moves, converged

In [19]:

solve_nqueens(solve_hillclimbing, size = 8,  ntrials = 1)

initial conflict count: 8.0
is converged: True
is valid solution obtained (check): True
number of iterations: 89
number of moves: 1288
display the results
[6, 1, 3, 0, 7, 4, 2, 5]
.  .  .  .  .  .  ♛  .
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  .  ♛
.  .  .  .  ♛  .  .  .
.  .  ♛  .  .  .  .  .
.  .  .  .  .  ♛  .  .



# Forward checking algorithm
Now implement your Forward Checking and check the values for num_iterations and number_moves for values of 10, 20 and 30. As this algorithm is not deterministic, run the algorithm 30 times for each of the values for n and report on the average values for the two metrics num_iterations and num_moves.

The code is very similar to dfs with backtracking except that instead of checking each column position in a new row for conflicts with existing rows, instead,  the conflicting columns in the ensuing rows are removed from consideration in the tree search.   

While backtracking, the conflicting columns must be reinstated but a single square on the board can have multiple conflicts from previous row placement.  Hence, during backtracking, the constraints are forward checked from scratch (a more efficient implementation would only roll back the new constraints introduced in the state to which the algorithm backtracks but that is future work).

First some helper functions to display the constrained boards

In [22]:
def displayConstraints(domains):
    if not columns:
        return
    n = len(domains)
    print(f"size: {n}")
    for domain in domains:
        rowStr = ["." if x in domain else "X" for x in range(n)]
        print("  ".join(rowStr))
    print()


def is_col_open_in_row(row, col, domains):
  return col in domains[row]

def forward_check(row, col, domains):
  for r, domain in enumerate(domains[row:], row):
    domain.discard(col) # remove same column
    domain.discard(abs(col-row+r)) # remove diagonal
    domain.discard(abs(col+row-r)) # remove anti diagonal

def forward_check_redo(size, columns):
  domains = [set(range(size)) for _ in range(size)]
  for row, col in enumerate(columns):
    for r, domain in enumerate(domains[row:], row):
      domain.discard(col) # remove same column
      domain.discard(abs(col-row+r)) # remove diagonal
      domain.discard(abs(col+row-r)) # remove anti diagonal
  return domains

def initialize_domains(size):
  assert size >= 0
  domains = [set(range(size)) for x in range(size)]
  return domains

size = 5
domains = initialize_domains(size)  # Initialize domains with all columns
print(domains)

print("initial board")
displayConstraints(domains)

forward_check(0, 0, domains)
print("board after placement (0,0)")
displayConstraints(domains)

forward_check(1, 3, domains)
print("board after placement (1,3)")
displayConstraints(domains)

columns = [0,3]
domains = forward_check_redo(size,columns)
print("board after redo for placements (0,0), (1,3)")
displayConstraints(domains)


[{0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}, {0, 1, 2, 3, 4}]
initial board
size: 5
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .
.  .  .  .  .

board after placement (0,0)
size: 5
X  .  .  .  .
X  X  .  .  .
X  .  X  .  .
X  .  .  X  .
X  .  .  .  X

board after placement (1,3)
size: 5
X  .  .  .  .
X  X  .  X  .
X  .  X  X  X
X  X  .  X  .
X  .  .  X  X

board after redo for placements (0,0), (1,3)
size: 5
X  .  .  .  .
X  X  .  X  .
X  .  X  X  X
X  X  .  X  .
X  .  .  X  X



In [25]:
def solve_forward_checking(size):
    columns = []
    domains = [set(range(size)) for _ in range(size)]  # Initialize domains with all columns
    number_of_moves = 0 #where do I change this so it counts the number of Queen moves?
    number_of_iterations = 0

    row = 0 # current row
    col = 0 # always start each row at leftmost col
    # iterate over rows of board
    while True:
        while col < size:
            number_of_iterations += 1
            if col in domains[row]:
                place_next_row(col, columns)
                forward_check(row, col, domains)
                number_of_moves += 1
                # print(f"placed: row: {row}, col: {col}")
                row += 1
                col = 0
                break
            else:
                # print(f"not placed: row: {row}, col: {col}")
                col += 1

        # could not find an open col in this row (backtrack),  or board is full
        if (col == size or row == size):
            number_of_iterations+=1
            # if board is full, we have a solution
            if row == size:
                print("I did it! Here is my solution")
                display(columns)
                converged = True
                return columns, number_of_iterations, number_of_moves, converged
            # else couldn't find a solution so need to backtrack
            # print("start to backtrack ... ")
            prev_col = remove_current_row(columns)
            if (prev_col == -1): # backtracked past column 1
                print("There are no solutions")
                #print(number_of_moves)
                converged = False
                return columns, number_of_iterations, number_of_moves, converged
            # retry previous row again, redo the forward checking
            row -= 1
            domains = forward_check_redo(size,columns)
            # start to now check at col = (1 + value of prev_column in the row)
            col = 1 + prev_col



In [26]:
solve_nqueens(solve_forward_checking, size = 8,  ntrials = 1)

I did it! Here is my solution


[1, 3, 5, 7, 2, 0, 6, 4]

is converged: True
is valid solution obtained (check): True
number of iterations: 1882
number of moves: 213
display the results
[1, 3, 5, 7, 2, 0, 6, 4]
.  ♛  .  .  .  .  .  .
.  .  .  ♛  .  .  .  .
.  .  .  .  .  ♛  .  .
.  .  .  .  .  .  .  ♛
.  .  ♛  .  .  .  .  .
♛  .  .  .  .  .  .  .
.  .  .  .  .  .  ♛  .
.  .  .  .  ♛  .  .  .



# Final Results

Challenge Question: Write a loop solving and printing each of the n-Queens problems to 40.  Can you get either of the solutions to get all the way to 40?  Did you need to add clever tricks to make it happen?